# Lightweight Fine-Tuning Project

In [ ]:
# Install the required version of datasets in case you have an older version
! pip install -q "datasets==2.15.0"

In [1]:
# Import the datasets and transformers packages

from datasets import load_dataset

# Load the train and test splits of the imdb dataset
splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset("imdb", split=splits))}

# Thin out the dataset to make it run faster
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(500))

# Show the dataset
ds

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 })}

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token_id = tokenizer.eos_token_id

def preprocess_function(examples):
    """Preprocess the imdb dataset by returning tokenized examples."""
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)


# Show the first example of the tokenized training set
print(tokenized_ds["train"][0]["input_ids"])

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

[1858, 318, 645, 8695, 379, 477, 1022, 6401, 959, 290, 4415, 5329, 475, 262, 1109, 326, 1111, 389, 1644, 2168, 546, 6590, 6741, 13, 4415, 5329, 3073, 42807, 11, 6401, 959, 3073, 6833, 13, 4415, 5329, 21528, 389, 2407, 2829, 13, 6401, 959, 338, 7110, 389, 1290, 517, 8253, 986, 6401, 959, 3073, 517, 588, 5537, 8932, 806, 11, 611, 356, 423, 284, 4136, 20594, 986, 383, 1388, 2095, 318, 4939, 290, 7650, 78, 11, 475, 423, 366, 27659, 40024, 590, 1911, 4380, 588, 284, 8996, 11, 284, 5052, 11, 284, 13446, 13, 1374, 546, 655, 13226, 30, 40473, 1517, 1165, 11, 661, 3597, 6401, 959, 3073, 1605, 475, 11, 319, 262, 584, 1021, 11, 11810, 484, 4702, 1605, 2168, 357, 10185, 737, 6674, 340, 338, 262, 3303, 11, 393, 262, 4437, 11, 475, 314, 892, 428, 2168, 318, 517, 3594, 621, 1605, 13, 2750, 262, 835, 11, 262, 10544, 389, 1107, 922, 290, 8258, 13, 383, 7205, 318, 407, 31194, 379, 477, 986, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50

In [24]:
import numpy as np
from transformers import AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


model = AutoModelForSequenceClassification.from_pretrained("gpt2",
      num_labels=2,
      id2label={0: "NEGATIVE", 1: "POSITIVE"},  # For converting predictions to strings
      label2id={"NEGATIVE": 0, "POSITIVE": 1},
    )
model.config.pad_token_id = tokenizer.pad_token_id
model

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

In [25]:


# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=20,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

#evaluate model prior to finetuning
trainer.evaluate()

{'eval_loss': 0.8299094438552856,
 'eval_accuracy': 0.51,
 'eval_runtime': 41.6712,
 'eval_samples_per_second': 11.999,
 'eval_steps_per_second': 3.0}

## Performing Parameter-Efficient Fine-Tuning

create a PEFT model from loaded model, run a training loop, and save the PEFT model weights.

In [38]:
from peft import LoraConfig
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.01,
    bias="none",
    task_type="SEQ_CLS"
)

In [39]:
from peft import get_peft_model
lora_model = get_peft_model(model, config)

In [40]:
lora_model.print_trainable_parameters()

trainable params: 294,912 || all params: 124,737,792 || trainable%: 0.2364255413467636


In [42]:


# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=20,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

#LORA finetuning
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.277733,0.892000
2,No log,0.368218,0.892000
3,No log,0.395359,0.886000
4,0.321300,0.477967,0.854000
5,0.321300,0.454512,0.884000
6,0.321300,0.536295,0.896000
7,0.321300,0.649111,0.878000
8,0.066900,0.764994,0.890000
9,0.066900,0.656012,0.898000
10,0.066900,0.699883,0.896000


TrainOutput(global_step=2500, training_loss=0.0875909015417099, metrics={'train_runtime': 3012.6008, 'train_samples_per_second': 3.319, 'train_steps_per_second': 0.83, 'total_flos': 5244148776960000.0, 'train_loss': 0.0875909015417099, 'epoch': 20.0})

In [43]:
lora_model.save_pretrained("gpt-lora")

## Performing Inference with a PEFT Model

load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Compare the results to the results from prior to fine-tuning.

In [46]:
from peft import AutoPeftModelForSequenceClassification
lora_model_trained = AutoPeftModelForSequenceClassification.from_pretrained("gpt-lora")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [48]:
lora_model_trained.config.pad_token_id = tokenizer.pad_token_id
trainer = Trainer(
    model=lora_model_trained,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

#LORA evaluate
trainer.evaluate()

{'eval_loss': 0.2777332067489624,
 'eval_accuracy': 0.892,
 'eval_runtime': 45.0288,
 'eval_samples_per_second': 11.104,
 'eval_steps_per_second': 1.399}

### Evaluation

In [52]:
print(f"Accuracy before finetuning with pretrained model: 51%\nAccuracy after finetuning with (PEFT)LORA: 89.2%")

Accuracy before finetuning with pretrained model: 51%
Accuracy after finetuning with (PEFT)LORA: 89.2%
